In this notebook, we run the stacked predictive scenario for each cognitive score and generate the prediction accuracies, as well as the weight contributions of each single-channel to the LASSO stacked model. As explained in the manuscript, out-of-sample predictions have been achieved using a Montecarlo cross-validation with 100 random splits into training (70% of the total data) and test (30% of the total data) set. 

In [1]:
import numpy as np
import pandas as pd

from os.path import join as opj
import sys
import time
import h5py

#Sklearn stuff
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import  (StandardScaler, PolynomialFeatures)
from sklearn.linear_model import (Lasso, LassoCV, LinearRegression)
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import mean_absolute_error, r2_score

from scipy.stats import ks_2samp

In [2]:
# add src folder from which we can import some customed functions
sys.path.append("/home/javier/Documentos/multimodal-cognition/")

# This is the stacking class
from stacking import StackingLassoCV

In [3]:
RANDOM_STATE = 0

Load the data

In [4]:
f =  h5py.File(opj("../data", "final_data.hdf5"), "a")
YY_domain_cognition =  f['YY_domain_cognition'][:]
X_conn = f['connectome_features'][:]
X_surf = f['surface_features'][:]
X_thic = f['thickness_features'][:]
X_subv = f['sub_vols_features'][:]
X_locc = f['loc_conn_features'][:]
subjects = f['subjects'][:]
f.close()

Load age data (restricted, so it's not in the repo) and binarise it in order to generate train/test partitions stratified by age.

In [6]:
age = pd.read_excel(opj("../data", "age.xlsx"))
age = pd.merge(pd.DataFrame({'Subject': subjects}), age, on='Subject')['Age_in_Yrs'].values

age_bin = np.digitize(age, np.quantile(age, q=np.arange(0.1, 1, 0.1)))

In [7]:
prepend_transformation = [VarianceThreshold(), StandardScaler(), PCA()]
stack_lasso = StackingLassoCV(prepend_transformation=prepend_transformation, cv=5,
                              n_jobs=1, random_state = RANDOM_STATE)

In [8]:
score_names = ['CogTotalComp_Unadj', 
               'CogFluidComp_Unadj', 
               'CogCrystalComp_Unadj', 
               'SCPT_SEN', 
               'DDisc_AUC_200', 
               'IWRD_TOT', 
               'VSPLOT_TC']

modality_names = ['CONNECTOME', 
                  'SURFACE', 
                  'THICKNESS', 
                  'SUB_VOLUMES', 
                  'LOCAL_CONN']
n_scores = len(score_names) 
n_mods = len(modality_names)

In [9]:
def adjust_age_new(YY_train, YY_test, M_train, M_test):
    
    """
    
    Function used to adjust age using a simple linear regression
    
    """
    
    # We already pass  M with an intercept, so we need to set this here to zero 
    linReg = LinearRegression(fit_intercept=False) 
    # Fit using ONLY training data
    linReg.fit(M_train, YY_train)
    
    # Substract predicted values and add intercept to keep the same scale. 
    # Here the intercept is the first column of the M matrix that we passed
    YY_train_adj = YY_train - linReg.predict(M_train) + linReg.coef_[:, 0]
    YY_test_adj = YY_test - linReg.predict(M_test) + linReg.coef_[:, 0]
    
    return YY_train_adj, YY_test_adj


In [10]:
def first_stage_predictions(list_X_train, list_X_test, y_train, y_test):
    
    """
    
    Function to generate the single-channel predictions to be
    later stacked and feed to a LASSO model
    
    """
    prepend_transformation = [VarianceThreshold(), StandardScaler(), PCA()]
    stack_lasso = StackingLassoCV(prepend_transformation=prepend_transformation, cv=5,
                                  n_jobs=1, random_state = RANDOM_STATE)
    
    stack_lasso.fit(list_X_train, y_train)

    part_train_cv = stack_lasso.stacked_features_

    y_stack_pred =  stack_lasso.predict(list_X_test)

    part_test_pred = y_stack_pred
        
    return part_train_cv, part_test_pred

In [11]:
def balanced(YY_train, YY_test):
    
    """
    
    Given a partition, check whether they are balanced 
    if the ditribution of the response variables in training and test set 
    don't differ as measured by a kolmogorosv-smirnov test.
    
    
    """
    
    ps = np.array([ks_2samp(YY_train[:,i], YY_test[:, i])[1] for i in range(YY_train.shape[1])])
    
    if np.all(ps > 0.05):
        return True
    else:
        return False

In [12]:
n_splits = 100 # Number of random splits in the Montecarlo cross-validation
n_train = int(0.7*X_conn.shape[0]) # Size of the training set
n_test = X_conn.shape[0] - n_train # Size of the test set
n_mods = len(modality_names) # Number of channels
n_scores = len(score_names) # Number of response variables

In [13]:
#Generate list of training/test set index partitions stratified by age

partition_idxs = []
# Construct single predictors classifiers
i_split = 0
seed = 0
while i_split < n_splits:
    
    # Generate partition
    sss = StratifiedShuffleSplit(n_splits=1, train_size=n_train, random_state=seed)
    train_index, test_index = list(sss.split(np.zeros(len(age_bin)), age_bin))[0]
    
    # Create training and test partitions
    YY_train = YY_domain_cognition[train_index]
    YY_test = YY_domain_cognition[test_index]
    
    # If training and test are not balanced, generate a new partition
    if balanced(YY_train, YY_test) is False:
        seed = seed + 1
        continue
    
    partition_idxs.append((train_index, test_index))
    #print("split %d finished\n" % i_split)
    i_split = i_split + 1
    seed = seed + 1

Store these partitions

In [39]:
with h5py.File(opj('../results', 'partitions.hdf5'), 'a') as f_partitions:
    f_partitions.create_dataset('training_idxs', 
                                data = np.array([train for (train, _) in  partition_idxs]).T)
    f_partitions.create_dataset('testing_idxs', 
                                data = np.array([test for (_, test) in  partition_idxs]).T)

In [49]:
# Construct single predictors classifiers
i_split = 0
seed = 0

single_r2_scores = np.zeros((n_splits, n_scores, n_mods))
comb_r2_scores = np.zeros((n_splits, n_scores))
comb_weights = np.zeros((n_splits, n_scores, n_mods))

#Create empty dictionary for the predictions
preds_dict = {}
for score in score_names:
    preds_dict[score] = []
    
poly =  PolynomialFeatures(degree = 1) 

#compute starting execution time
start = time.time()    
for train_index, test_index in partition_idxs:

    YY_train = YY_domain_cognition[train_index]
    YY_test = YY_domain_cognition[test_index]
    
    age_train, age_test = age[train_index], age[test_index] 
    M_train = poly.fit_transform(age_train[:, np.newaxis]) 
    M_test = poly.transform(age_test[:, np.newaxis]) 
    
    # Adjust by  age 
    YY_train_adj, YY_test_adj = adjust_age_new(YY_train, YY_test, M_train, M_test)
            
    for jj, score in enumerate(score_names):
            
        y_train = YY_train_adj[:, jj]
        y_test = YY_test_adj[:, jj]
        
        # Compute single predictions
        X_train_2, X_test_2 = first_stage_predictions([X_conn[train_index], 
                                                       X_surf[train_index], 
                                                       X_thic[train_index], 
                                                       X_subv[train_index], 
                                                       X_locc[train_index]], 
                                                      [X_conn[test_index], 
                                                       X_surf[test_index], 
                                                       X_thic[test_index], 
                                                       X_subv[test_index], 
                                                       X_locc[test_index]], y_train, y_test)     
        
        # Store single channel r2
        single_r2_scores[i_split, jj,:] = np.array([r2_score(y_test, X_test_2[:, kk]) \
                                                  for kk in range(n_mods)])
        
        # Define Lasso second classifier
        clf_2 = LassoCV(cv=5, random_state=RANDOM_STATE)
        
        # Fit on the stacked predictions
        clf_2.fit(X_train_2, y_train)
        
        # predict on the stacked predictions
        y_pred_comb = clf_2.predict(X_test_2)
        
        # Compute and store r2 for stacked predictions
        r2_comb = r2_score(y_test, y_pred_comb)
        comb_r2_scores[i_split, jj] = r2_comb
        
        # Store the weights for each single prediction
        comb_weights[i_split, jj, :] = clf_2.coef_
        
        #Create dataframe with the predictions
        preds_df = pd.DataFrame({'y_pred_conn': X_test_2[:, 0],
                                 'y_pred_surf': X_test_2[:, 1],
                                 'y_pred_thic': X_test_2[:, 2],
                                 'y_pred_subv': X_test_2[:, 3],
                                 'y_pred_locc': X_test_2[:, 4],
                                 'y_pred_comb': y_pred_comb,
                                 'y_test': y_test}
                    )
        
        # Store this
        preds_dict[score].append(preds_df)
    
    print("split %d finished" % i_split)
    i_split = i_split + 1

# compute final execution time and elapsed time
done = time.time()
elapsed_time = done - start

split 0 finished
split 1 finished
split 2 finished
split 3 finished
split 4 finished
split 5 finished
split 6 finished
split 7 finished
split 8 finished
split 9 finished
split 10 finished
split 11 finished
split 12 finished
split 13 finished
split 14 finished
split 15 finished
split 16 finished
split 17 finished
split 18 finished
split 19 finished
split 20 finished
split 21 finished
split 22 finished
split 23 finished
split 24 finished
split 25 finished
split 26 finished
split 27 finished
split 28 finished
split 29 finished
split 30 finished
split 32 finished
split 33 finished
split 34 finished
split 35 finished
split 36 finished
split 37 finished
split 38 finished
split 39 finished
split 40 finished
split 41 finished
split 42 finished
split 43 finished
split 44 finished
split 45 finished
split 46 finished
split 47 finished
split 48 finished
split 49 finished
split 50 finished
split 51 finished
split 52 finished
split 53 finished
split 54 finished
split 55 finished
split 56 finished
sp

Store these results into hdf5 files

In [50]:
with h5py.File(opj('../results', 'predictions.hdf5'), 'a') as f_preds:

    for score, preds in preds_dict.items():
        f_preds.create_dataset(score, 
                               data = np.asarray([mc_pred.to_numpy()
                                                  for mc_pred in preds]))

In [51]:
f_res = h5py.File(opj('../results', 'scores.hdf5'), 'a')
f_res.create_dataset('comb_2_scores', data=comb_r2_scores)
f_res.create_dataset('single_r2_scores', data=single_r2_scores)
f_res.create_dataset('comb_weights', data = comb_weights)
f_res.close()